In [15]:
def crawl_twitter(tweet_list, days_range, start, end, query_search, max_tweets):
    import requests
    from bs4 import BeautifulSoup

    def get_bs_obj(url):
        result = requests.get(url)
        bs_obj = BeautifulSoup(result.content, "html.parser")

        return bs_obj

    try:
        import GetOldTweets3 as got
    except:
        !pip install GetOldTweets3
        import GetOldTweets3 as got

    import datetime

    start = datetime.datetime.strptime(start, "%Y-%m-%d")
    end = datetime.datetime.strptime(end, "%Y-%m-%d")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

    for date in date_generated:
        days_range.append(date.strftime("%Y-%m-%d"))

    print("=== 설정된 트윗 수집 기간은 {} 에서 {} 까지 입니다 ===".format(days_range[0], days_range[-1]))
    print("=== 총 {}일 간의 데이터 수집 ===".format(len(days_range)))


    import time

    # 수집 기간 맞추기
    start_date = days_range[0]
    end_date = (datetime.datetime.strptime(days_range[-1], "%Y-%m-%d") 
                + datetime.timedelta(days=1)).strftime("%Y-%m-%d") # setUntil이 끝을 포함하지 않으므로, day + 1

    # 트윗 수집 기준 정의
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query_search)\
                                               .setSince(start_date)\
                                               .setUntil(end_date)\
                                               .setMaxTweets(max_tweets)

    # 수집 with GetOldTweet3
    print("Collecting data start.. from {} to {}".format(days_range[0], days_range[-1]))
    start_time = time.time()

    tweet = got.manager.TweetManager.getTweets(tweetCriteria)

    print("Collecting data end.. {0:0.2f} Minutes".format((time.time() - start_time)/60))
    print("=== Total num of tweets is {} ===".format(len(tweet)))

    from random import uniform
    from tqdm import tqdm_notebook

    for index in tqdm_notebook(tweet):

        # 메타데이터 목록 
        username = index.username
        link = index.permalink 
        content = index.text

      # 결과 합치기
        info_list = [ username, content, link]
        tweet_list.append(info_list)

        # 휴식 
        time.sleep(uniform(1,2))

In [16]:
def save_csv(tweet_list, days_range):
    import pandas as pd

    twitter_df = pd.DataFrame(tweet_list, 
                              columns = ["user_name", "text", "link"])

    # csv 파일 만들기
    twitter_df.to_csv("영화_twitter_data_{}_to_{}.csv".format(days_range[0], days_range[-1]), index=False)
    print("=== {} tweets are successfully saved ===".format(len(tweet_list)))

    df_tweet = pd.read_csv('영화_twitter_data_{}_to_{}.csv'.format(days_range[0], days_range[-1]))
    df_tweet.head(10)

In [18]:
days_range = []
tweet_list = []

crawl_twitter(tweet_list, days_range, "2020-05-01", "2020-05-22", '문재인', 10000)
save_csv(tweet_list, days_range)

=== 설정된 트윗 수집 기간은 2020-05-01 에서 2020-05-21 까지 입니다 ===
=== 총 21일 간의 데이터 수집 ===
=== Total num of tweets is 10000 ===


C:\Users\shadow98a\anaconda3\lib\site-packages\ipykernel_launcher.py:55: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



=== 10000 tweets are successfully saved ===
